<a href="https://colab.research.google.com/github/kathdevx/emotionalAI/blob/main/fine_tune_wav2vec2_base_960h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 311 kB 28.1 MB/s 
     |████████████████████████████████| 133 kB 75.1 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 1.1 MB 63.9 MB/s 
     |████████████████████████████████| 243 kB 74.2 MB/s 
     |████████████████████████████████| 271 kB 77.7 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 78.3 MB/s 
     |████████████████████████████████| 3.5 MB 30.0 MB/s 
     |████████████████████████████████| 6.8 MB 54.5 MB/s 
     |████████████████████████████████| 895 kB 59.9 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from datasets import load_dataset, load_metric
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor
import os
from typing import Tuple
from transformers.file_utils import ModelOutput
import librosa
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch
import shutil
import transformers
from datasets import Dataset
import numpy as np
from transformers import EvalPrediction
import pandas as pd

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
split_set = ['Train', 'Validation', 'Test1', 'Test2']

for split in split_set:
    split_df = pd.read_csv('/content/drive/MyDrive/Data/'+split+'.csv')
    split_df['Path'] = split_df['Path'].str.replace('Podcast Data/Audios', '/content/drive/MyDrive/Data/Audio Files')
    new_len_60 = round(len(split) * 0.6)
    split_df_60 = split_df[:new_len_60]
    split_df_60 = split_df_60[['FileName', 'EmoClass', 'Path']].copy()
    new_fname = f'/content/drive/MyDrive/Data/new_{split}60.csv'
    split_df_60.to_csv(new_fname, index=False)

In [ ]:
split_df = pd.read_csv('/content/drive/MyDrive/Data/new_Train60.csv')
split_df.head()

,FileName,EmoClass,Path
0,MSP-PODCAST_0001_0049.wav,X,/content/drive/MyDrive/Data/Audio Files/MSP-PO...
1,MSP-PODCAST_0001_0066.wav,N,/content/drive/MyDrive/Data/Audio Files/MSP-PO...
2,MSP-PODCAST_0001_0087.wav,N,/content/drive/MyDrive/Data/Audio Files/MSP-PO...
3,MSP-PODCAST_0001_0179.wav,F,/content/drive/MyDrive/Data/Audio Files/MSP-PO...


In [ ]:
split_df = None

In [ ]:
data_files = {
    "train": "/content/drive/MyDrive/Data/new_Train60.csv",
    "validation": "/content/drive/MyDrive/Data/new_Validation60.csv"
}

dataset = load_dataset("csv", data_files=data_files, delimiter=",", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

input_column = "Path"
output_column = "EmoClass"
# output_columns = ["EmoAct","EmoVal", "EmoDom"]

label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)

Using custom data configuration default-adf1bbf1d356c40c


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-adf1bbf1d356c40c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import IPython.display as ipd
    
speech, sr = torchaudio.load(train_dataset[1]['Path'])
speech = speech[0].numpy().squeeze()
speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

In [ ]:
model_name_or_path = 'facebook/wav2vec2-base-960h'
pooling_mode = 'mean'


config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
    local_files_only=False,
    problem_type="single_label_classification"
)

setattr(config, 'pooling_mode', pooling_mode)

processor = Wav2Vec2Processor.from_pretrained(model_name_or_path, return_attention_mask=True)
target_sampling_rate = processor.feature_extractor.sampling_rate

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
def speech_file_to_array_fn(path):
    
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()

    return speech


def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label


def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]    


    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=3
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=3
)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:166: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:166: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:166: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
idx = 0
print(f"Training input_values: {train_dataset[idx]['input_values']}")
print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['EmoClass']}")

Training input_values: [-0.4638236463069916, -0.3821949362754822, -0.2461470663547516, -0.12176046520471573, -0.014865724369883537, 0.05510174110531807, 0.0900854766368866, 0.10466203093528748, 0.11923858523368835, 0.15422232449054718, 0.19698022305965424, 0.22418980300426483, 0.26403236389160156, 0.3155362010002136, 0.3223385810852051, 0.3349716067314148, 0.3339998424053192, 0.3087337911128998, 0.2766653895378113, 0.2397381216287613, 0.20864146947860718, 0.19989553093910217, 0.19503667950630188, 0.17268596589565277, 0.1853189766407013, 0.1483917087316513, 0.11049265414476395, 0.05218643322587013, -0.011950413696467876, -0.0012609391706064343, -0.0459623783826828, -0.06636955589056015, -0.09455089271068573, -0.12176046520471573, -0.1684054434299469, -0.2092197984457016, -0.24711883068084717, -0.2859896719455719, -0.30250975489616394, -0.3481829762458801, -0.38802555203437805, -0.3909408748149872, -0.4220375120639801, -0.436614066362381, -0.4414729177951813, -0.4579930305480957, -0.4725

In [ ]:
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
is_regression = False

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [ ]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config
)

model.freeze_feature_extractor()

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'wav2vec2.masked_spec_embed', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a

In [ ]:

from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="/content/model_output60",
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    gradient_accumulation_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=50,
    learning_rate=1e-4,
    save_total_limit=2,
    metric_for_best_model='loss',
    load_best_model_at_end=True
)

from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()



trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=20)]
)

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# list_of_files = glob.glob('/content/model_output1/check*')
# latest_checkpoint = max(list_of_files, key=os.path.getctime)
torch.cuda.empty_cache()
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: Path, EmoClass, FileName.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4
  Num Epochs = 50
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 1000
  Gradient Accumulation steps = 10
  Total optimization steps = 50


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: Path, EmoClass, FileName.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 100


RuntimeError: ignored

In [ ]:
%cp -av '/content/model_output3' '/content/drive/MyDrive/Data/model_output_lr2'

'/content/model_output3' -> '/content/drive/MyDrive/Data/model_output_lr2'
'/content/model_output3/runs' -> '/content/drive/MyDrive/Data/model_output_lr2/runs'
'/content/model_output3/runs/Feb20_20-37-54_c4053cd424a5' -> '/content/drive/MyDrive/Data/model_output_lr2/runs/Feb20_20-37-54_c4053cd424a5'
'/content/model_output3/runs/Feb20_20-37-54_c4053cd424a5/events.out.tfevents.1645389479.c4053cd424a5.82.13' -> '/content/drive/MyDrive/Data/model_output_lr2/runs/Feb20_20-37-54_c4053cd424a5/events.out.tfevents.1645389479.c4053cd424a5.82.13'
'/content/model_output3/runs/Feb20_20-37-54_c4053cd424a5/1645389479.628904' -> '/content/drive/MyDrive/Data/model_output_lr2/runs/Feb20_20-37-54_c4053cd424a5/1645389479.628904'
'/content/model_output3/runs/Feb20_20-37-54_c4053cd424a5/1645389479.628904/events.out.tfevents.1645389479.c4053cd424a5.82.14' -> '/content/drive/MyDrive/Data/model_output_lr2/runs/Feb20_20-37-54_c4053cd424a5/1645389479.628904/events.out.tfevents.1645389479.c4053cd424a5.82.14'
'/co